# 02 - Exploratory Data Analysis
##### In this notebook, we:
##### 1. explore the availability of the discrete sampling locations and times for the CMC and CBP datasets
##### 2. evaluate data augmentation
##### 3. perform preliminary statistical analyses (correlation matrix)
##### 4. plot time series

In [2]:
#load modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request

#install geopandas through cluster page: libraries->pypi search->"geopandas"
import geopandas 

#time series graphing
%matplotlib inline
import seaborn as sns
import datetime as dt
import scipy.stats as stats
from io import StringIO

In [3]:
# File location and type
file_location = "/FileStore/tables/eda_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Create a view or table

temp_table_name = "eda_data_csv"
df.createOrReplaceTempView(temp_table_name)
datadf = df.toPandas()

### 1. Explore the availability of the discrete sampling locations and times for the CMC and CBP datasets
In working with our community partner, the Chesapeake Monitoring Cooperative, we are incorporating water quality sampling efforts by citizen scientists. However, the data are sparse. We add sampling data from the Chesapeake Bay Program, a regional government and non-profit partnership for restoration of the Chesapeake Bay.

Plot a map of the CMC and CBP station locations. Are any regions better represented than others?

In [6]:
# make a geo data frame including the points of all the station locations
gdf=geopandas.GeoDataFrame(datadf,
                           geometry=geopandas.points_from_xy(
                             datadf.Longitude.astype(float),datadf.Latitude.astype(float)))

In [7]:
# import a map of US states
fid=urllib.request.urlretrieve("https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json",
                               "/tmp/gz_2010_us_040_00_500k.json") 
us_states = geopandas.read_file(fid[0])

In [8]:
# separate out the CMC vs CBP sample locations
gdf_cmc = gdf.loc[gdf.SampleId.astype(float)<3]
gdf_cbp = gdf.loc[gdf.SampleId.astype(float)>=3]

In [9]:
# make a map of sampling locations

plt.rcParams.update({'font.size': 16})

ax = us_states.boundary.plot(color='black',figsize=[7,14])

gdf_cbp.plot(ax=ax, color='blue')
gdf_cmc.plot(ax=ax, color='red')


minx, miny, maxx, maxy = gdf.total_bounds
ax.set_xlim(minx-1, maxx+1)
ax.set_ylim(miny-1, maxy+1)
ax.set_title('CMC (red) & CBP (blue) sampling locations \n N=30,321')

plt.show()

Temporal: Plot a time series of the data. Are certain years or seasons more highly sampled than others?

In [11]:
# plot a random sampling of 1000 points from the data 
import random
plt.rcParams.update({'font.size': 14})
datadf.Date = pd.to_datetime(datadf.Date)
inds=random.sample(range(1,len(datadf.Date)),1000)
plt.plot(datadf.Date[inds],datadf.tn.astype(float)[inds],'.')
plt.xlabel('Time')
plt.ylabel('Total Nitrogen (mg/L)')

Dataset insights:
* There appear to be more sampling points around Maryland compared to Virginia, Pennsylvania, Delaware, and New York. 
* CMC data points are more clustered and CBP data points are more widely distributed.
* There are more data points in recent years exhibiting greater variability in TN

### 2. Evaluate data augmentation
Due to the sparseness of climate data in discrete observations, we augment the climate variables with data from a leading historical climate data product, the North American Regional Reanalysis. How does it compare to the in-situ observations from the CMC dataset?

In [14]:
x=datadf.airtemp.astype(float)
y=datadf.airtemp_narr.astype(float)

plt.xlim(-5, y.max())
plt.ylim(-5, y.max())
plt.plot(x, y,'o',color = 'k')

plt.title('CMC and NARR air temp comparison, N=193')
plt.xlabel('CMC air temp')
plt.ylabel('NARR air temp')
plt.show()

In [15]:
#number of data points included in this comparison
print(x[~np.isnan(x)].count())

193

There is a bias between the two measurements, but they appear to correlate well.

### 3. Preliminary statistics: Correlation Matrix

In [18]:
# apply correlation anlysis to the entire dataset, not just per station 
datadf.apply(lambda x: x.factorize()[0]).corr()

,Date,Time,StationName,StationCode,Latitude,Longitude,GroupCode,SampleId,SampleDepth,airtemp,do,tn,tp,watertemp,RainfallWithin24Hours,RainfallWithin48Hours,Rainfall,WindSpeed,airtemp_narr,precip3_narr,precip24_narr,precip48_narr,windspeed_narr,HUC12,year,month,geometry
Date,1.000000,0.030512,0.817255,0.407391,0.401301,0.403642,0.441633,0.812846,0.266703,-0.105115,0.236353,0.279752,0.114983,0.298974,0.440492,0.439381,0.440621,0.439484,0.800929,0.140984,0.409218,0.534808,0.811619,0.359881,-0.296207,0.137033,0.403642
Time,0.030512,1.000000,0.043850,-0.013093,-0.013237,-0.011920,-0.075689,0.008227,-0.006739,-0.039725,-0.170927,0.100281,0.012437,-0.142193,0.121382,0.121215,0.121485,0.122755,0.006040,0.024409,0.007517,0.009241,0.009405,-0.008907,0.063247,0.011634,-0.011920
StationName,0.817255,0.043850,1.000000,0.526587,0.519049,0.522425,0.586528,0.903251,0.336553,-0.139206,0.280326,0.323515,0.143227,0.342225,0.596866,0.595679,0.596996,0.595749,0.879271,0.154645,0.423500,0.583235,0.896730,0.460869,-0.226211,0.098316,0.522425
StationCode,0.407391,-0.013093,0.526587,1.000000,0.988044,0.988146,0.586875,0.473915,0.299110,-0.114392,0.185015,0.316482,0.163108,0.147441,0.526104,0.525774,0.526039,0.525721,0.467044,0.098257,0.245434,0.325423,0.473157,0.900605,-0.093597,0.080550,0.988146
Latitude,0.401301,-0.013237,0.519049,0.988044,1.000000,0.999938,0.575746,0.465839,0.295602,-0.114859,0.179512,0.313965,0.164627,0.141707,0.521177,0.520839,0.521115,0.520796,0.458888,0.097286,0.242927,0.321745,0.464991,0.909904,-0.089957,0.079481,0.999938
Longitude,0.403642,-0.011920,0.522425,0.988146,0.999938,1.000000,0.580316,0.467540,0.298994,-0.114905,0.180339,0.314630,0.165257,0.142867,0.528693,0.528363,0.528630,0.528314,0.460628,0.097754,0.243809,0.322977,0.466760,0.909507,-0.086766,0.080539,1.000000
GroupCode,0.441633,-0.075689,0.586528,0.586875,0.575746,0.580316,1.000000,0.475869,0.445255,-0.137348,0.420469,0.251589,0.120404,0.382135,0.730603,0.731086,0.729567,0.729967,0.472972,0.086556,0.229264,0.319817,0.477927,0.551691,0.042025,0.106000,0.580316
SampleId,0.812846,0.008227,0.903251,0.473915,0.465839,0.467540,0.475869,1.000000,0.258256,-0.099695,0.269522,0.318077,0.130565,0.338927,0.414631,0.413548,0.414786,0.413638,0.974871,0.143440,0.446274,0.625476,0.997422,0.415757,-0.477978,0.069321,0.467540
SampleDepth,0.266703,-0.006739,0.336553,0.299110,0.295602,0.298994,0.445255,0.258256,1.000000,-0.105761,0.202766,0.149207,0.137306,0.214259,0.478821,0.478176,0.478876,0.477353,0.258445,0.051865,0.120513,0.172648,0.260689,0.248983,0.039681,0.084072,0.298994
airtemp,-0.105115,-0.039725,-0.139206,-0.114392,-0.114859,-0.114905,-0.137348,-0.099695,-0.105761,1.000000,-0.009686,-0.058388,-0.037280,-0.031714,-0.241113,-0.239932,-0.240452,-0.233640,-0.098788,-0.023154,-0.051106,-0.069057,-0.100220,-0.099668,-0.046263,-0.054397,-0.114905


Correlation matrix for the whole dataset

In [20]:
f = plt.figure(figsize=(19, 15))
plt.matshow(datadf.apply(lambda x: x.factorize()[0]).corr(), fignum=f.number)
plt.xticks(range(datadf.shape[1]), datadf.columns, fontsize=14, rotation=90)
plt.yticks(range(datadf.shape[1]), datadf.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
#plt.title('Correlation Matrix', fontsize=16)

Date is highly correlated with airtemp and windspeed; windspeed and air temp are highly correlated. These are capturing a temporal pattern in weather.

In [22]:
#Select station with the most data and within the long/late ranges
datadf['StationCode'].value_counts().idxmax()

Out[12]: 'TF1.2'

In [23]:
# do correlation based on a specific station
nwa_wec = datadf.loc[datadf['StationCode'] == 'TF1.2'] 

# remove null values 'RainfallWithin24Hours'
# 'RainfallWithin48Hours',
# 'Rainfall',
# 'WindSpeed',
#  'airtemp',
#narrow to look at correlations that are not repetitive 
nwa_wec_narrowed = nwa_wec[[ 'SampleDepth',
 'Date',
 'do',
 'tn',
 'tp',
 'watertemp',
 'airtemp_narr',
 'precip3_narr',
 'precip24_narr',
 'precip48_narr',
 'windspeed_narr']]

# apply factorization so that categorical data can also be included (will encode the string to numeric values)
nwa_wec_narrowed.apply(lambda x: x.factorize()[0]).corr() 

,SampleDepth,Date,do,tn,tp,watertemp,airtemp_narr,precip3_narr,precip24_narr,precip48_narr,windspeed_narr
SampleDepth,1.000000,0.398916,0.157967,0.303049,0.329639,0.159446,0.398999,0.088558,0.367407,0.393608,0.398945
Date,0.398916,1.000000,0.198569,0.872166,0.691999,0.400667,0.999960,0.285447,0.687966,0.895848,0.999996
do,0.157967,0.198569,1.000000,0.177403,0.157798,0.178970,0.198959,0.041547,0.165429,0.214776,0.198625
tn,0.303049,0.872166,0.177403,1.000000,0.591724,0.346198,0.872124,0.236912,0.557583,0.762290,0.872255
tp,0.329639,0.691999,0.157798,0.591724,1.000000,0.337920,0.692056,0.291472,0.546898,0.636215,0.691934
watertemp,0.159446,0.400667,0.178970,0.346198,0.337920,1.000000,0.400405,0.199494,0.299850,0.376832,0.400887
airtemp_narr,0.398999,0.999960,0.198959,0.872124,0.692056,0.400405,1.000000,0.285301,0.688261,0.895784,0.999964
precip3_narr,0.088558,0.285447,0.041547,0.236912,0.291472,0.199494,0.285301,1.000000,0.346267,0.341574,0.285708
precip24_narr,0.367407,0.687966,0.165429,0.557583,0.546898,0.299850,0.688261,0.346267,1.000000,0.779077,0.688042
precip48_narr,0.393608,0.895848,0.214776,0.762290,0.636215,0.376832,0.895784,0.341574,0.779077,1.000000,0.895862


Correlation matrix for station TF1.2, which has the most data points

In [25]:
f = plt.figure(figsize=(19, 15))
plt.matshow(nwa_wec_narrowed.apply(lambda x: x.factorize()[0]).corr() , fignum=f.number)
plt.xticks(range(nwa_wec_narrowed.shape[1]), nwa_wec_narrowed.columns, fontsize=14, rotation=45)
plt.yticks(range(nwa_wec_narrowed.shape[1]), nwa_wec_narrowed.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)

TN is fairly highly correlated with date, airtemp and wind speed

### 4. Time Series Graphs for total nitrogen for all data and for just TF1.2

In [28]:
datadf["tn"] = pd.to_numeric(datadf["tn"])
datadf.plot(x='Date', y='tn',  title='Total Nitrogen in all Stations',
            style='.',alpha=.5,ylim=(0,30))
plt.ylabel('mg/L')

In [29]:
# To perform the linear regression we need the dates to be numeric
nwa_wec_narrowed.index = nwa_wec_narrowed.Date.map(dt.date.toordinal)
nwa_wec_narrowed.tn=nwa_wec_narrowed.tn.astype(float)
# Perform linear regression
slope, y0, r, p, stderr = stats.linregress(nwa_wec_narrowed.index, 
                                           nwa_wec_narrowed['tn'].astype(float))

# x co-ordinates for the start and end of the line
x_endpoints = pd.DataFrame([nwa_wec_narrowed.index[0], nwa_wec_narrowed.index[-1]])

# Compute predicted values from linear regression
y_endpoints = y0 + slope * x_endpoints

# Subset just to date and TN
ts=nwa_wec_narrowed[['Date', 'tn']]

# Overlay the line
ax=ts.plot(x='Date', y ='tn', label = 'tn', title='Total Nitrogen in Station TF1.2')
ax.plot(x_endpoints, y_endpoints, 'k-', label="Trend Line")
plt.gca().legend(('Total Nitrogen','Trend Line'))
plt.ylabel('mg/L')

In [30]:
nwa_wec_narrowed.plot(x='Date', y='tn', style='o', title='Total Nitrogen (Scattered) in Station TF1.2')
plt.ylabel('mg/L')

Time Series of climate variables from NARR

In [32]:
# air temp: plot a random sampling of 5000 points from the data 
plt.rcParams.update({'font.size': 14})
inds=random.sample(range(1,len(datadf.Date)),5000)
plt.plot(datadf.Date[inds],datadf.airtemp_narr.astype(float)[inds],'.')
plt.xlabel('Time')
plt.ylabel('Air Temp [deg C]')
plt.title('NARR Air Temp at CMC & CBP measurement stations')

In [33]:
#precipitation: a random sampling of 5000 points
plt.rcParams.update({'font.size': 14})
inds=random.sample(range(1,len(datadf.Date)),5000)
plt.plot(datadf.Date[inds],datadf.precip48_narr.astype(float)[inds],'o',label='48 hrs prior')
plt.plot(datadf.Date[inds],datadf.precip24_narr.astype(float)[inds],'o',label='24 hrs prior',alpha=.5)
plt.plot(datadf.Date[inds],datadf.precip3_narr.astype(float)[inds],'o',label='3 hrs prior',alpha=.5)
plt.xlabel('Time')
plt.ylabel('accumulated precipitation')
plt.title('Precipitation CMC & CBP measurement stations')
plt.legend()